In [2]:
import numpy as np
from zipfile import ZipFile
import pickle
import re
import torch
from torch import nn, optim
from scipy.spatial import distance
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load data

!wget -O a.txt "https://drive.google.com/uc?export=download&id=1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56"
!wget -O x.zip "https://drive.google.com/uc?export=download&id=1221858Ognjuffo3Pv6QezSiBw8xBOBe-"
!wget -O equal.pth "https://drive.google.com/uc?export=download&id=1_RIKfYvv6AuBmpeHZv1Nb3e5_qgpxRh0"

a = np.loadtxt('a.txt')
with ZipFile('/content/x.zip', 'r') as f:
  f.extract('x.txt')

x_real = []
x_imag = []
x = []
with open('x.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      x.append((temp.real, temp.imag))
      x_real.append(temp.real)
      x_imag.append(temp.imag)

y_real = []
y_imag = []
#with open('drive/MyDrive/y.txt') as input_file:
with open('y.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      y_real.append(temp.real)
      y_imag.append(temp.imag)

--2023-02-01 08:35:22--  https://drive.google.com/uc?export=download&id=1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56
Resolving drive.google.com (drive.google.com)... 142.251.163.100, 142.251.163.101, 142.251.163.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.163.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9l0kai12jpulj7o1sgnop1ujdlmt6k2j/1675240500000/00806258749903615801/*/1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56?e=download&uuid=7dfb5742-4fc7-444a-b79a-ac3c8675e13e [following]
--2023-02-01 08:35:24--  https://doc-14-80-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9l0kai12jpulj7o1sgnop1ujdlmt6k2j/1675240500000/00806258749903615801/*/1u5TAN-nRjHoAQ02J5-3o3Yr53m66Vw56?e=download&uuid=7dfb5742-4fc7-444a-b79a-ac3c8675e13e
Resolving doc-14-80-docs.googleusercontent.com (doc-14-80-docs.googleusercontent.com)... 172.253.63.132, 2

In [4]:
# Reshape

y_real_t = torch.Tensor(y_real).reshape(-1,1)
y_imag_t = torch.Tensor(y_imag).reshape(-1,1)
Y = torch.stack((y_real_t, y_imag_t), -1)

In [5]:
# Define nonlinear NN for equalizer

class NonlinEq(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(2, 50)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(50, 20)
        self.relu2 = nn.ReLU()
        self.lin3 = nn.Linear(20,2)

    def forward(self, y):
        y = self.relu1(self.lin1(y))
        y = self.relu2(self.lin2(y))
        return self.lin3(y)[::2]

In [6]:
# Convert y_eq in two arrays so that it can be plot

nleq = torch.load('equal.pth')
y_eq = nleq(Y)
yeq = y_eq.detach().numpy()
yeq=yeq[:,0,:]
yeq_real = yeq[:,0]
yeq_imag = yeq[:,1]

In [7]:
# Create a list with the 16 different mapped symbols

map_sym = list(dict.fromkeys(x))

# Test the euclidean distance between one point equalized and every point of the constellation 

A_hat = []
X_hat = []

for j in range(yeq_imag.size):
    min_dst = 10
    for k in range(len(map_sym)):
        b = (yeq_real[j], yeq_imag[j]) # equalized point
        c = map_sym[k] # constellation point
        dst = distance.euclidean(b, c)

        # Choose the point with minimum euclidean distance
        if min_dst > dst: 
            min_dst = dst
            a_hat = a[x.index(c)] 
            x_hat = c

    # List with the hard decision made
    A_hat.append(a_hat)
    X_hat.append(x_hat)

In [ ]:
# Save result

with open("A_hat", "wb") as fp:   
    pickle.dump(A_hat, fp)
with open("X_hat", "wb") as fp:   
    pickle.dump(X_hat, fp)